In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/regularization-techniques/test.csv
/kaggle/input/regularization-techniques/train.csv


In [3]:
df_train= pd.read_csv('/kaggle/input/regularization-techniques/train.csv')
df_test = pd.read_csv('/kaggle/input/regularization-techniques/test.csv')

In [4]:
train_values=df_train.values
test_values = df_test.values

In [5]:
X_train = np.array(train_values[:,1:])
Y_train = np.array(train_values[:,0])
X_test = np.array(test_values[:,1:])
Y_test = np.array(test_values[:,0])

**Data Augumentation (Rescaling)**

In [6]:
X_train=X_train.reshape(-1,28,28,1)/255.0
X_train.shape

(27455, 28, 28, 1)

In [7]:
X_test = X_test.reshape(-1,28,28,1)/255.0
X_test.shape

(7172, 28, 28, 1)

**Importing Libraries**

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from keras.utils.np_utils import to_categorical 


In [9]:
number_of_labels = np.max(Y_train)
number_of_labels

24

 **Model without regularization techniques**

In [10]:
def create_model(INPUT_SHAPE):
    model = Sequential()
    model.add(Conv2D(filters = 8, kernel_size=(3, 3),strides=(1,1),padding="valid", input_shape=INPUT_SHAPE))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2),strides=(2,2),padding="same"))

    model.add(Conv2D(filters = 16, kernel_size=(3, 3),strides=(1,1),padding="valid"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2),strides=(2,2),padding="same"))

    model.add(Conv2D(filters = 32, kernel_size=(3, 3),strides=(1,1),padding="valid"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(number_of_labels+1, activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    model.summary()
    return model
    

In [11]:
X_train[0].shape

(28, 28, 1)

In [12]:
model=create_model(X_train[0].shape)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 8)         80        
_________________________________________________________________
batch_normalization (BatchNo (None, 26, 26, 8)         32        
_________________________________________________________________
activation (Activation)      (None, 26, 26, 8)         0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 8)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 16)        1168      
_________________________________________________________________
batch_normalization_1 (Batch (None, 11, 11, 16)        64        
_________________________________________________________________
activation_1 (Activation)    (None, 11, 11, 16)        0

In [13]:
to_categorical(Y_train,number_of_labels+1).shape

(27455, 25)

In [14]:
model.fit(X_train,to_categorical(Y_train,number_of_labels+1),batch_size=32,epochs=5,validation_split=0.2)

Epoch 1/5
687/687 [==============================] - 4s 5ms/step - loss: 0.6504 - accuracy: 0.7932 - val_loss: 0.0347 - val_accuracy: 0.9927
Epoch 2/5
687/687 [==============================] - 3s 4ms/step - loss: 0.0462 - accuracy: 0.9861 - val_loss: 0.0060 - val_accuracy: 0.9989
Epoch 3/5
687/687 [==============================] - 3s 4ms/step - loss: 0.0160 - accuracy: 0.9957 - val_loss: 0.7242 - val_accuracy: 0.8192
Epoch 4/5
687/687 [==============================] - 3s 5ms/step - loss: 0.0349 - accuracy: 0.9903 - val_loss: 0.0064 - val_accuracy: 0.9989
Epoch 5/5
687/687 [==============================] - 3s 4ms/step - loss: 0.0011 - accuracy: 0.9998 - val_loss: 6.3784e-04 - val_accuracy: 0.9998


In [15]:
loss,accuracy=model.evaluate(X_test,to_categorical(Y_test,number_of_labels+1))
print("Accuracy of the model without regularization: "+ str(accuracy))

225/225 [==============================] - 1s 2ms/step - loss: 0.5970 - accuracy: 0.8833
Accuracy of the model without regularization: 0.8832961320877075


L1 Regularization

In [16]:
from keras import regularizers

In [17]:
def create_model_with_L1(INPUT_SHAPE):
    model = Sequential()
    model.add(Conv2D(filters = 8, kernel_size=(3, 3),strides=(1,1),padding="valid", input_shape=INPUT_SHAPE))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2),strides=(2,2),padding="same"))

    model.add(Conv2D(filters = 16, kernel_size=(3, 3),strides=(1,1),padding="valid"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2),strides=(2,2),padding="same"))

    model.add(Conv2D(filters = 32, kernel_size=(3, 3),strides=(1,1),kernel_regularizer=regularizers.l2(0.01),padding="valid"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    model.add(Flatten())
    model.add(Dense(128,kernel_regularizer=regularizers.l1(0.01), activation='relu'))
    model.add(Dense(128,kernel_regularizer=regularizers.l1(0.01), activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64,kernel_regularizer=regularizers.l1(0.01), activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(32,kernel_regularizer=regularizers.l1(0.01), activation='relu'))
    model.add(Dense(number_of_labels+1, activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    model.summary()
    return model

In [18]:
model=create_model_with_L1(X_train[0].shape)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 26, 26, 8)         80        
_________________________________________________________________
batch_normalization_3 (Batch (None, 26, 26, 8)         32        
_________________________________________________________________
activation_3 (Activation)    (None, 26, 26, 8)         0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 8)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 11, 11, 16)        1168      
_________________________________________________________________
batch_normalization_4 (Batch (None, 11, 11, 16)        64        
_________________________________________________________________
activation_4 (Activation)    (None, 11, 11, 16)       

In [19]:
model.fit(X_train,to_categorical(Y_train,number_of_labels+1),batch_size=32,epochs=5,validation_split=0.2)

Epoch 1/5
687/687 [==============================] - 3s 5ms/step - loss: 6.5969 - accuracy: 0.0456 - val_loss: 3.2842 - val_accuracy: 0.0419
Epoch 2/5
687/687 [==============================] - 3s 5ms/step - loss: 3.2824 - accuracy: 0.0464 - val_loss: 3.2816 - val_accuracy: 0.0441
Epoch 3/5
687/687 [==============================] - 3s 5ms/step - loss: 3.2803 - accuracy: 0.0441 - val_loss: 3.2796 - val_accuracy: 0.0479
Epoch 4/5
687/687 [==============================] - 3s 5ms/step - loss: 3.2801 - accuracy: 0.0439 - val_loss: 3.2803 - val_accuracy: 0.0406
Epoch 5/5
687/687 [==============================] - 3s 5ms/step - loss: 3.2797 - accuracy: 0.0442 - val_loss: 3.2796 - val_accuracy: 0.0479


In [20]:
loss,accuracy=model.evaluate(X_test,to_categorical(Y_test,number_of_labels+1))
print("Accuracy of the model with L1 Regularization: "+ str(accuracy))

225/225 [==============================] - 1s 3ms/step - loss: 3.3034 - accuracy: 0.0201
Accuracy of the model with L1 Regularization: 0.020078081637620926


***L2 Regularization with Dropout***

In [21]:
def create_model_with_L2_Dropout(INPUT_SHAPE):
    model = Sequential()
    model.add(Conv2D(filters = 8, kernel_size=(3, 3),strides=(1,1),padding="valid", input_shape=INPUT_SHAPE))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2),strides=(2,2),padding="same"))

    model.add(Conv2D(filters = 16, kernel_size=(3, 3),strides=(1,1),padding="valid"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2),strides=(2,2),padding="same"))

    model.add(Conv2D(filters = 32, kernel_size=(3, 3),strides=(1,1),kernel_regularizer=regularizers.l2(0.01),padding="valid"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    
    model.add(Flatten())
    model.add(Dense(128,kernel_regularizer=regularizers.l2(0.01), activation='relu'))
    model.add(Dense(128,kernel_regularizer=regularizers.l2(0.01), activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64,kernel_regularizer=regularizers.l2(0.01), activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(32,kernel_regularizer=regularizers.l2(0.01), activation='relu'))
    model.add(Dense(number_of_labels+1, activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    model.summary()
    return model

In [22]:
model=create_model_with_L2_Dropout(X_train[0].shape)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 26, 26, 8)         80        
_________________________________________________________________
batch_normalization_6 (Batch (None, 26, 26, 8)         32        
_________________________________________________________________
activation_6 (Activation)    (None, 26, 26, 8)         0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 13, 13, 8)         0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 11, 11, 16)        1168      
_________________________________________________________________
batch_normalization_7 (Batch (None, 11, 11, 16)        64        
_________________________________________________________________
activation_7 (Activation)    (None, 11, 11, 16)       

In [23]:
model.fit(X_train,to_categorical(Y_train,number_of_labels+1),batch_size=32,epochs=5,validation_split=0.2)

Epoch 1/5
687/687 [==============================] - 4s 5ms/step - loss: 2.6574 - accuracy: 0.3922 - val_loss: 1.3134 - val_accuracy: 0.7164
Epoch 2/5
687/687 [==============================] - 3s 5ms/step - loss: 1.2290 - accuracy: 0.7360 - val_loss: 0.8542 - val_accuracy: 0.8579
Epoch 3/5
687/687 [==============================] - 4s 6ms/step - loss: 0.8987 - accuracy: 0.8595 - val_loss: 0.6021 - val_accuracy: 0.9636
Epoch 4/5
687/687 [==============================] - 3s 5ms/step - loss: 0.7689 - accuracy: 0.9009 - val_loss: 0.5022 - val_accuracy: 0.9863
Epoch 5/5
687/687 [==============================] - 4s 5ms/step - loss: 0.6593 - accuracy: 0.9247 - val_loss: 0.4601 - val_accuracy: 0.9871


In [24]:
loss,accuracy=model.evaluate(X_test,to_categorical(Y_test,number_of_labels+1))
print("Accuracy of the model with L2 regularization and Dropout: "+ str(accuracy))

225/225 [==============================] - 1s 3ms/step - loss: 0.8687 - accuracy: 0.8769
Accuracy of the model with L2 regularization and Dropout: 0.8768823146820068


**Model with L1,L2,Dropout and EarlyStopping**

In [25]:
def create_model_with_L1_L2_Dropout_EarlyStopping(INPUT_SHAPE):
    model = Sequential()
    model.add(Conv2D(filters = 8, kernel_size=(3, 3),strides=(1,1),padding="valid", input_shape=INPUT_SHAPE))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2),strides=(2,2),padding="same"))

    model.add(Conv2D(filters = 16, kernel_size=(3, 3),strides=(1,1),padding="valid"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2),strides=(2,2),padding="same"))

    model.add(Conv2D(filters = 32, kernel_size=(3, 3),strides=(1,1),kernel_regularizer=regularizers.l1(0.01),padding="valid"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    
    model.add(Flatten())
    model.add(Dense(128,kernel_regularizer=regularizers.l1(0.01), activation='relu'))
    model.add(Dense(128,kernel_regularizer=regularizers.l2(0.01), activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64,kernel_regularizer=regularizers.l2(0.01), activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(32,kernel_regularizer=regularizers.l2(0.01), activation='relu'))
    model.add(Dense(number_of_labels+1, activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    model.summary()
    return model

In [26]:
model=create_model_with_L1_L2_Dropout_EarlyStopping(X_train[0].shape)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 26, 26, 8)         80        
_________________________________________________________________
batch_normalization_9 (Batch (None, 26, 26, 8)         32        
_________________________________________________________________
activation_9 (Activation)    (None, 26, 26, 8)         0         
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 13, 13, 8)         0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 11, 11, 16)        1168      
_________________________________________________________________
batch_normalization_10 (Batc (None, 11, 11, 16)        64        
_________________________________________________________________
activation_10 (Activation)   (None, 11, 11, 16)       

In [27]:
from keras.optimizers import RMSprop,SGD,Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint = ModelCheckpoint(r'C:\Label.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)

earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=3,
                          verbose=1,
                          restore_best_weights=True
                          )

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=3,
                              verbose=1,
                              min_delta=0.0001)

callbacks = [earlystop,checkpoint,reduce_lr]

In [28]:
model.fit(X_train,to_categorical(Y_train,number_of_labels+1),
          batch_size=32,epochs=10,
          validation_split=0.2,
          callbacks=callbacks)

Epoch 1/10
682/687 [============================>.] - ETA: 0s - loss: 5.0783 - accuracy: 0.1031
Epoch 00001: val_loss improved from inf to 3.02185, saving model to C:\Label.h5
687/687 [==============================] - 4s 6ms/step - loss: 5.0627 - accuracy: 0.1034 - val_loss: 3.0218 - val_accuracy: 0.1064
Epoch 2/10
676/687 [============================>.] - ETA: 0s - loss: 2.5879 - accuracy: 0.1681
Epoch 00002: val_loss improved from 3.02185 to 2.38188, saving model to C:\Label.h5
687/687 [==============================] - 4s 5ms/step - loss: 2.5859 - accuracy: 0.1685 - val_loss: 2.3819 - val_accuracy: 0.1991
Epoch 3/10
682/687 [============================>.] - ETA: 0s - loss: 2.4151 - accuracy: 0.1957
Epoch 00003: val_loss improved from 2.38188 to 2.22159, saving model to C:\Label.h5
687/687 [==============================] - 4s 5ms/step - loss: 2.4145 - accuracy: 0.1963 - val_loss: 2.2216 - val_accuracy: 0.2388
Epoch 4/10
682/687 [============================>.] - ETA: 0s - loss: 2

In [29]:
loss,accuracy=model.evaluate(X_test,to_categorical(Y_test,number_of_labels+1))
print("Accuracy of the model with L2 , L1 and Dropout with earlyStopping and ReduceLrOnPlateau: "+ str(accuracy))

225/225 [==============================] - 1s 3ms/step - loss: 2.2687 - accuracy: 0.3009
Accuracy of the model with L2 , L1 and Dropout with earlyStopping and ReduceLrOnPlateau: 0.30089235305786133


In [30]:
X_tra=X_train.reshape(-1,784)
X_tes=X_test.reshape(-1,784)

**AdaBoost Classifier**

In [31]:
from sklearn.ensemble import AdaBoostClassifier
model = AdaBoostClassifier(random_state=1)
model.fit(X_tra,Y_train)


AdaBoostClassifier(random_state=1)

In [32]:
model.score(X_tes,Y_test)

0.31553262688232014

**XGBoost**

In [ ]:
from xgboost import XGBClassifier
model_xgb = XGBClassifier()
model_xgb.fit(X_tra, Y_train)
model_xgb.score(X_tes,Y_test)